# Global M. bovis lineages - fetch data

* Brites paper: https://open-research-europe.ec.europa.eu/articles/1-100/v2
* Joe data: https://www.ncbi.nlm.nih.gov/bioproject/PRJNA523164

In [1]:
import sys,os,shutil,subprocess
import glob,random
from importlib import reload
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
import pylab as plt
import matplotlib as mpl
from Bio import SeqIO
from snipgenie import clustering

## Brites data

In [85]:
#brites = pd.read_csv('brites_table1.csv')
brites = pd.read_csv('brites_table1.txt',sep='\t').iloc[:,:9]
brites = brites.dropna(subset=['Accession_Number'])
spmap = {'La2':'M.caprae','La3':'M.orygis'}
brites['Species'] = brites['Clonal_Complex'].map(spmap).fillna('M.bovis')
#brites=brites[brites.Clonal_Complex!='L6']
brites.to_csv('brites_table1.csv',index=False)

c=brites.Country_Isolation.value_counts()
countries=list(c[-24:].index)
brites.Clonal_Complex.value_counts()

Clonal_Complex
La1.8.1       261
La1.7.1       119
La1.2         105
La3            77
La1.3          64
La2            61
La1.7-unk4     31
La1.8.2        30
La1.5          12
La1.6           9
La1.1           9
La1.7-unk5      9
La1.4           7
La1.8-unk6      7
L6              1
La1.8-unk8      1
Name: count, dtype: int64

In [53]:
brites = brites[~brites.Country.isin(['Ireland','United Kingdom'])]
#low quality or v large
omit = ['ERR017796','SRR7131025','SRR7131117','SRR6797355','SRR10251186','SRR7617446','SRR7851314','SRR1792178',
        'SRR4199188','ERR2512942','ERR3458087','SRR2101329','SRR5642718','ERR2517102']

## fetch data

In [54]:
def fetch_data(meta,col,path,omit=[]):
    for i,row in meta.iterrows():
        run = row[col]
        #print (row)
        if run in omit:
            continue
        if 'ERS' in run:
            continue
        files = glob.glob(os.path.join(path,'%s*.fastq*') %run)
        if len(files)>0:
            continue
        cmd = 'fastq-dump --split-files -O %s %s' %(path,run)
        print (run)
        print (cmd)
        try:
            subprocess.check_output(cmd, shell=True)
        except:
            print ('failed')
            continue
        f=os.path.join(path,'%s*.fastq' %run)
        cmd = 'pigz %s' %f
        print (cmd)
        subprocess.check_output(cmd, shell=True)
        

In [ ]:
path = '/storage/btbgenie/brites'
#rare = brites[brites.Country.isin(countries)]
sub = brites.sample(40)
fetch_data(sub,'Accession_Number',path,omit)

In [6]:
'''files = glob.glob('/storage/btbgenie/mbovis_sra/*.fastq*')
for f in files:
    name=os.path.splitext(os.path.basename(f))[0].split('_')[0]
    #print (name)
    if name not in list(meta.Accession_Number):
        print (name,os.path.basename(f))
        shutil.move(f,os.path.join('/storage/btbgenie/mbovis_sra_other',os.path.basename(f)))'''

In [84]:
samples = pd.read_csv('/storage/btbgenie/global_results/samples.csv')
x=brites[brites.Accession_Number.isin(samples['sample'])]
x.Country_Isolation.value_counts()
samples[~samples['sample'].isin(brites.Accession_Number)]

,index,sample,filename1,filename2,name1,name2,read_length,bam_file,mapped,reads,perc_mapped,coverage,meandepth,missing_sites
424,ERR1082113,ERR1082113,/storage/btbgenie/brites/ERR1082113_1.fastq.gz,/storage/btbgenie/brites/ERR1082113_2.fastq.gz,ERR1082113_1,ERR1082113_2,125,/storage/btbgenie/global_results/mapped/ERR108...,7684664.0,3875768.0,99.14,99.59,219.72,NaN


## Joe data

In [6]:
ukmeta = pd.read_csv('/storage/btbgenie/UK/joe_runtable.txt')

path='/storage/btbgenie/UK'
#fetch_data(ukmeta,'Run',path)

In [7]:
#check against brites for dups
ukmeta.Run
brites[brites.Accession_Number.isin(ukmeta.Run)]

,Accession_Number,Clonal_Complex,Country,Gnumber,Host,Map,SIB_Number,Sample.included.in.Loiseau.et.al..2020
